In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
# extract a line start with 'mse'
def extract_mse(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in reversed(lines):
            if line.startswith('mse'):
                return line
    return None

In [3]:
# extract the previous line of the last line start with 'Validation'
def extract_val(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in reversed(lines):
            if line.startswith('Validation'):
                line=lines[lines.index(line) - 1]
                float(re.findall(r"mse:\d+\.?\d*", line)[0][4:])
                return lines[lines.index(line) - 1]
    return None

In [4]:
# extract the float number behind 'mse:'
def extract_float(line):
    return float(re.findall(r"mse:\d+\.?\d*", line)[0][4:])

In [5]:
# find the last line start with 'Validation'
def extract_last_val(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in reversed(lines):
            if line.startswith('Validation'):
                line=float(re.findall(r"--> \d+\.?\d*", line)[0][4:])
                return line
    return None

In [6]:
# find the line start with 'FLOPs' and 'params', extract the following numbers
def extract_flops_params(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        flops=0
        params=0
        for line in lines:
            if line.startswith('FLOPs'):
                flops = float(re.findall(r"\d+\.?\d*", line)[0])
            if line.startswith('params'):
                params = float(re.findall(r"\d+\.?\d*", line)[0])
    return flops, params

In [7]:
# find the last line with 'Saving model ...' before the first line with '1 out of 3'
def extract_last_save(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        scaned=[]
        for line in lines:
            if line.startswith('Epoch:'):
                scaned.append(line)
            if line.startswith('EarlyStopping counter: 1 out of 3'):
                # return scaned[-3]
                # get the number after 'Test Loss: '
                return float(re.findall(r"Test Loss: \d+\.?\d*", scaned[-3])[0][10:])
    return None

In [9]:
extract_float(extract_mse('./2_FITS_Ettm1_720_720_H14_s1919.log'))

0.414766401052475

In [8]:
import os
file_list = os.listdir('./')


In [9]:
file_list

['1j_FITS_Weather_720_96_H10_bs32.log',
 '1j_FITS_Weather_720_192_H10_bs32.log',
 '1j_FITS_Weather_720_336_H10_bs32.log',
 '1j_FITS_Weather_720_720_H10_bs32.log',
 '1j_FITS_Weather_720_96_H10_bs64.log',
 '1j_FITS_Weather_720_192_H10_bs64.log',
 '1j_FITS_Weather_720_336_H10_bs64.log',
 '1j_FITS_Weather_720_720_H10_bs64.log',
 '1j_FITS_Weather_720_96_H10_bs128.log',
 '1j_FITS_Weather_720_192_H10_bs128.log',
 '1j_FITS_Weather_720_336_H10_bs128.log',
 '1j_FITS_Weather_720_720_H10_bs128.log',
 '1j_FITS_Weather_720_96_H10_bs256.log',
 '1j_FITS_Weather_720_192_H10_bs256.log',
 '1j_FITS_Weather_720_336_H10_bs256.log',
 '1j_FITS_Weather_720_720_H10_bs256.log',
 '2j_FITS_Weather_720_96_H10_bs32.log',
 '2j_FITS_Weather_720_192_H10_bs32.log',
 '2j_FITS_Weather_720_336_H10_bs32.log',
 '2j_FITS_Weather_720_720_H10_bs32.log',
 '2j_FITS_Weather_720_96_H10_bs64.log',
 '2j_FITS_Weather_720_192_H10_bs64.log',
 '2j_FITS_Weather_720_336_H10_bs64.log',
 '2j_FITS_Weather_720_720_H10_bs64.log',
 '2j_FITS_Weat

In [11]:
datasets = ['Etth2']
# datasets = ['weather', 'Exchange','Traffic','Electricity'] 'Ettm1','Ettm2',
train_mode = ['1','2']
Batchsize=['32', '64']
H_order=['10']
output_len=['96','192','336','720']

In [12]:
writer=pd.ExcelWriter('./FITS_weather.xlsx')
for output in output_len:
    if output == '96':
        startrow=0
    elif output == '192':
        startrow=6
    elif output == '336':
        startrow=12
    elif output == '720':
        startrow=18
    a=pd.DataFrame(columns=['1','2'],index=H_order)
    for bs in Batchsize:
        for H in H_order:
            results=[]
            file_name = '1j_FITS_Weather_720_'+output+'_H'+H+'_bs'+bs+'.log'
            if file_name in file_list:
                print(file_name)
                line = extract_mse(file_name)
                if line:
                    result=extract_float(line)
                    results.append(result)
                    # print(extract_last_val(file_name))
                else:
                    print('None')
                # print(extract_last_val(file_name))
            # get the mean and std of the results
            if results:
                a.loc[H,bs]=str(round(np.mean(results),3))+'±'+str(round(np.std(results),3))
        
    a.to_excel(writer,sheet_name='mode1',startcol=0,startrow=startrow)

for output in output_len:
    if output == '96':
        startrow=0
    elif output == '192':
        startrow=6
    elif output == '336':
        startrow=12
    elif output == '720':
        startrow=18
    a=pd.DataFrame(columns=['1','2'],index=H_order)
    for bs in Batchsize:
        for H in H_order:
            results=[]
            file_name = '2j_FITS_Weather_720_'+output+'_H'+H+'_bs'+bs+'.log'
            if file_name in file_list:
                print(file_name)
                line = extract_mse(file_name)
                if line:
                    result=extract_float(line)
                    results.append(result)
                    # print(extract_last_val(file_name))
                else:
                    print('None')
                # print(extract_last_val(file_name))
            # get the mean and std of the results
            if results:
                a.loc[H,bs]=str(round(np.mean(results),3))+'±'+str(round(np.std(results),3))
        
    a.to_excel(writer,sheet_name='mode2',startcol=0,startrow=startrow)

writer.save()

1j_FITS_Weather_720_96_H10_bs32.log
1j_FITS_Weather_720_96_H10_bs64.log
1j_FITS_Weather_720_192_H10_bs32.log
1j_FITS_Weather_720_192_H10_bs64.log
1j_FITS_Weather_720_336_H10_bs32.log
1j_FITS_Weather_720_336_H10_bs64.log
1j_FITS_Weather_720_720_H10_bs32.log
1j_FITS_Weather_720_720_H10_bs64.log
2j_FITS_Weather_720_96_H10_bs32.log
2j_FITS_Weather_720_96_H10_bs64.log
2j_FITS_Weather_720_192_H10_bs32.log
2j_FITS_Weather_720_192_H10_bs64.log
2j_FITS_Weather_720_336_H10_bs32.log
2j_FITS_Weather_720_336_H10_bs64.log
2j_FITS_Weather_720_720_H10_bs32.log
2j_FITS_Weather_720_720_H10_bs64.log


In [43]:
for dataset in datasets:
    for output in output_len:
        if output == '96':
            startrow=0
        elif output == '192':
            startrow=6
        elif output == '336':
            startrow=12
        elif output == '720':
            startrow=18
        for input in input_len:
            a=pd.DataFrame(columns=['0','1','2'],index=H_order)
            for mode in train_mode:
                for H in H_order:
                    file_name = mode+'_FLinear_'+dataset+'_'+input+'_'+output+'_H'+H+'.log'
                    if file_name in file_list:
                        print(file_name)
                        line = extract_mse(file_name)
                        if line:
                            result=extract_float(line)
                            a.loc[H,mode]=result
                            # print(extract_last_val(file_name))
                        else:
                            print('None')
            if input == '90':
                a.to_excel(writer,sheet_name=dataset,startcol=0,startrow=startrow)
            elif input == '180':
                a.to_excel(writer,sheet_name=dataset,startcol=4,startrow=startrow)
            elif input == '360':
                a.to_excel(writer,sheet_name=dataset,startcol=8,startrow=startrow)
            elif input == '720':
                a.to_excel(writer,sheet_name=dataset,startcol=12,startrow=startrow)

writer.save()



0_FLinear_Ettm1_90_96_H4.log
0_FLinear_Ettm1_90_96_H6.log
0_FLinear_Ettm1_90_96_H8.log
0_FLinear_Ettm1_90_96_H10.log
0_FLinear_Ettm1_90_96_H12.log
None
1_FLinear_Ettm1_90_96_H4.log
1_FLinear_Ettm1_90_96_H6.log
1_FLinear_Ettm1_90_96_H8.log
1_FLinear_Ettm1_90_96_H10.log
1_FLinear_Ettm1_90_96_H12.log
2_FLinear_Ettm1_90_96_H4.log
2_FLinear_Ettm1_90_96_H6.log
2_FLinear_Ettm1_90_96_H8.log
2_FLinear_Ettm1_90_96_H10.log
2_FLinear_Ettm1_90_96_H12.log
0_FLinear_Ettm1_180_96_H4.log
0_FLinear_Ettm1_180_96_H6.log
0_FLinear_Ettm1_180_96_H8.log
0_FLinear_Ettm1_180_96_H10.log
0_FLinear_Ettm1_180_96_H12.log
1_FLinear_Ettm1_180_96_H4.log
1_FLinear_Ettm1_180_96_H6.log
1_FLinear_Ettm1_180_96_H8.log
1_FLinear_Ettm1_180_96_H10.log
1_FLinear_Ettm1_180_96_H12.log
2_FLinear_Ettm1_180_96_H4.log
2_FLinear_Ettm1_180_96_H6.log
2_FLinear_Ettm1_180_96_H8.log
2_FLinear_Ettm1_180_96_H10.log
2_FLinear_Ettm1_180_96_H12.log
0_FLinear_Ettm1_360_96_H4.log
0_FLinear_Ettm1_360_96_H6.log
0_FLinear_Ettm1_360_96_H8.log
0_FLinea

In [46]:
datasets = ['Ettm1','Ettm2']
# datasets = ['weather', 'Exchange','Traffic','Electricity'] ,
train_mode = ['1','2']
input_len=['90','180','360','720']
H_order=['4','6','8','10','12']
output_len=['96','192','336','720']

In [47]:
for dataset in datasets:
    for output in output_len:
        if output == '96':
            startrow=0
        elif output == '192':
            startrow=6
        elif output == '336':
            startrow=12
        elif output == '720':
            startrow=18
        for input in input_len:
            a=pd.DataFrame(columns=['0','1','2'],index=H_order)
            for mode in train_mode:
                for H in H_order:
                    file_name = mode+'_FLinear_'+dataset+'_'+input+'_'+output+'_H'+H+'.log'
                    if file_name in file_list:
                        print(file_name)
                        line = extract_mse(file_name)
                        if line:
                            result=extract_float(line)
                            a.loc[H,mode]=result
                            # print(extract_last_val(file_name))
                        else:
                            print('None')
                        if mode=='2':
                            result0=extract_last_save(file_name)
                            a.loc[H,'0']=result0
            if input == '90':
                a.to_excel(writer,sheet_name=dataset,startcol=0,startrow=startrow)
            elif input == '180':
                a.to_excel(writer,sheet_name=dataset,startcol=4,startrow=startrow)
            elif input == '360':
                a.to_excel(writer,sheet_name=dataset,startcol=8,startrow=startrow)
            elif input == '720':
                a.to_excel(writer,sheet_name=dataset,startcol=12,startrow=startrow)

writer.save()

1_FLinear_Ettm1_90_96_H4.log
1_FLinear_Ettm1_90_96_H6.log
1_FLinear_Ettm1_90_96_H8.log
1_FLinear_Ettm1_90_96_H10.log
1_FLinear_Ettm1_90_96_H12.log
2_FLinear_Ettm1_90_96_H4.log
2_FLinear_Ettm1_90_96_H6.log
2_FLinear_Ettm1_90_96_H8.log
2_FLinear_Ettm1_90_96_H10.log
2_FLinear_Ettm1_90_96_H12.log
1_FLinear_Ettm1_180_96_H4.log
1_FLinear_Ettm1_180_96_H6.log
1_FLinear_Ettm1_180_96_H8.log
1_FLinear_Ettm1_180_96_H10.log
1_FLinear_Ettm1_180_96_H12.log
2_FLinear_Ettm1_180_96_H4.log
2_FLinear_Ettm1_180_96_H6.log
2_FLinear_Ettm1_180_96_H8.log
2_FLinear_Ettm1_180_96_H10.log
2_FLinear_Ettm1_180_96_H12.log
1_FLinear_Ettm1_360_96_H4.log
1_FLinear_Ettm1_360_96_H6.log
1_FLinear_Ettm1_360_96_H8.log
1_FLinear_Ettm1_360_96_H10.log
1_FLinear_Ettm1_360_96_H12.log
2_FLinear_Ettm1_360_96_H4.log
2_FLinear_Ettm1_360_96_H6.log
2_FLinear_Ettm1_360_96_H8.log
2_FLinear_Ettm1_360_96_H10.log
2_FLinear_Ettm1_360_96_H12.log
1_FLinear_Ettm1_720_96_H4.log
1_FLinear_Ettm1_720_96_H6.log
1_FLinear_Ettm1_720_96_H8.log
1_FLinea

In [21]:
writer=pd.ExcelWriter('./FLinear_flops.xlsx')
for dataset in datasets:
    for output in output_len:
        if output == '96':
            startrow=0
        elif output == '192':
            startrow=6
        elif output == '336':
            startrow=12
        elif output == '720':
            startrow=18
        for input in input_len:
            a=pd.DataFrame(columns=['FLOPs','Params'],index=['15','30','45','60','75'])
            for freq in cut_freq:
                file_name = '0_FLinear_'+dataset+'_'+input+'_'+output+'_c'+freq+'.log'
                if file_name in file_list:
                    print(file_name)
                    try:
                        flop, param = extract_flops_params(file_name)
                    except:
                        pass
                    a.loc[freq,'FLOPs']=flop
                    a.loc[freq,'Params']=param
                else:
                    print('None')
                

            if input == '90':
                a.to_excel(writer,sheet_name=dataset,startcol=0,startrow=startrow)
            elif input == '180':
                a.to_excel(writer,sheet_name=dataset,startcol=4,startrow=startrow)
            elif input == '336':
                a.to_excel(writer,sheet_name=dataset,startcol=8,startrow=startrow)
            elif input == '360':
                a.to_excel(writer,sheet_name=dataset,startcol=12,startrow=startrow)
            elif input == '700':
                a.to_excel(writer,sheet_name=dataset,startcol=16,startrow=startrow)

writer.save()

None
None
None
None
None
0_FLinear_Ettm1_180_96_c15.log
0_FLinear_Ettm1_180_96_c30.log
0_FLinear_Ettm1_180_96_c45.log
0_FLinear_Ettm1_180_96_c60.log
None
0_FLinear_Ettm1_336_96_c15.log
0_FLinear_Ettm1_336_96_c30.log
0_FLinear_Ettm1_336_96_c45.log
0_FLinear_Ettm1_336_96_c60.log
None
0_FLinear_Ettm1_360_96_c15.log
0_FLinear_Ettm1_360_96_c30.log
0_FLinear_Ettm1_360_96_c45.log
0_FLinear_Ettm1_360_96_c60.log
None
0_FLinear_Ettm1_700_96_c15.log
0_FLinear_Ettm1_700_96_c30.log
0_FLinear_Ettm1_700_96_c45.log
0_FLinear_Ettm1_700_96_c60.log
None
None
None
None
None
None
0_FLinear_Ettm1_180_192_c15.log
0_FLinear_Ettm1_180_192_c30.log
0_FLinear_Ettm1_180_192_c45.log
0_FLinear_Ettm1_180_192_c60.log
None
0_FLinear_Ettm1_336_192_c15.log
0_FLinear_Ettm1_336_192_c30.log
0_FLinear_Ettm1_336_192_c45.log
0_FLinear_Ettm1_336_192_c60.log
None
0_FLinear_Ettm1_360_192_c15.log
0_FLinear_Ettm1_360_192_c30.log
0_FLinear_Ettm1_360_192_c45.log
0_FLinear_Ettm1_360_192_c60.log
None
0_FLinear_Ettm1_700_192_c15.log
0_F